# House price prediction with mlflow

In [1]:
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.datasets import fetch_california_housing

In [2]:
housing_df = fetch_california_housing()

In [3]:
housing_df

{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
           37.88      , -122.23      ],
        [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
           37.86      , -122.22      ],
        [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
           37.85      , -122.24      ],
        ...,
        [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
           39.43      , -121.22      ],
        [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
           39.43      , -121.32      ],
        [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
           39.37      , -121.24      ]], shape=(20640, 8)),
 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894], shape=(20640,)),
 'frame': None,
 'target_names': ['MedHouseVal'],
 'feature_names': ['MedInc',
  'HouseAge',
  'AveRooms',
  'AveBedrms',
  'Population',
  'AveOccup',
  'Latitude',
  'Longitude'],
 'DESCR': 

In [4]:
# Preparing the dataset

df = pd.DataFrame(housing_df.data,columns = housing_df.feature_names )

In [5]:
df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


In [6]:
df['Price']  = housing_df.target

# Train test split, Model Hyperparameter Tining and MLFlow Experiments

In [7]:
from urllib.parse import urlparse

In [8]:
df.columns

Index(['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup',
       'Latitude', 'Longitude', 'Price'],
      dtype='object')

In [9]:
df.shape

(20640, 9)

In [10]:
X = df.drop(columns=["Price"])
y = df['Price']

In [11]:
X_train, X_test, y_train, y_test  = train_test_split(X,y,test_size=0.20,random_state=42)

In [12]:
def hyperparameter_tuning(X_train,y_train,param_grid):
    rf_model = RandomForestRegressor()
    grid  = GridSearchCV(estimator=rf_model,param_grid=param_grid,cv=3,n_jobs=-1,verbose=2,scoring = "neg_mean_squared_error")
    grid.fit(X_train,y_train)
    return grid
    

In [13]:
from mlflow.models import infer_signature
infer_signature(X_train,y_train)

# Define hyperparameter grid

param_grid = {
    'n_estimators':[100,200],
    'max_depth':[5,10,None],
    'min_samples_split': [2,5],
    'min_samples_leaf': [1,2]
}
# Start Mlflow experiments
with mlflow.start_run():
    # perform hyperparameter_tuning
    grid_search  = hyperparameter_tuning(X_train,y_train,param_grid)

    # Get best model
    best_model = grid_search.best_estimator_

    #evaluate the model
    y_pred = best_model.predict(X_test)
    mse = mean_squared_error(y_test,y_pred)

    # Log best parameter and metrics
    mlflow.log_param("best_n_estimators", grid_search.best_params_['n_estimators'])
    mlflow.log_param("best_max_depth", grid_search.best_params_['max_depth'])
    mlflow.log_param("best_min_samples_split", grid_search.best_params_['min_samples_split'])
    mlflow.log_param("best_min_samples_leaf", grid_search.best_params_['min_samples_leaf'])
    mlflow.log_metric("mse",mse)

    #Tracking url
    mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")
    tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

    if tracking_url_type_store != 'file':
        mlflow.sklearn.log_model(best_model,"model",registered_model_name="Best RF model")
    else:
        mlflow.sklearn.load_model(best_model,"model",signature = signature)
    print(f"Best Hyperparameters: {grid_search.best_params_}")
    print(f"Mean_Squared Error: {mse}")
    

Fitting 3 folds for each of 24 candidates, totalling 72 fits


2025/11/06 01:54:15 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/06 01:54:18 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'Best RF model'.
2025/11/06 01:54:20 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Best RF model, version 1
Created version '1' of model 'Best RF model'.


Best Hyperparameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Mean_Squared Error: 0.2527948982442377
🏃 View run youthful-ox-852 at: http://127.0.0.1:5000/#/experiments/0/runs/2e52c2e7e84e42608bf1182da98e68fb
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/0
